ПРАКТИКА ПО СЛУЧАЙНОМУ ЛЕСУ

In [517]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [518]:
df = pd.read_csv('weatherAUS.csv')
df.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No


In [519]:
#df.info()

In [520]:
df.isnull().sum().sum()

343248

In [521]:
null_features = list(df.isnull().mean()[df.isnull().mean() >= 0.4].index) #Получим список признаков с >= 40% пропусков
df = df.drop(null_features, axis=1)
df.shape

(145460, 20)

In [522]:
set(df['RainToday'].values)

{'No', 'Yes', nan}

In [523]:
def set_bool_rainfeatures(x):
    if x == 'Yes':
        return 1
    elif x == 'No':
        return 0
    else:
        return x
    
df['RainToday'] = df['RainToday'].apply(set_bool_rainfeatures)
df['RainTomorrow'] = df['RainTomorrow'].apply(set_bool_rainfeatures)

In [524]:
df['RainToday'].mean().round(2)

0.22

In [525]:
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month


In [526]:
#Разделим покоордиратно суммарные осадки в месяц на количество разных дней с наблюдениями в этом месяце

month_rain = df.groupby(['Month'])['Rainfall'].sum() / df.groupby(['Month'])['Date'].nunique()
month_rain.sort_values(ascending=False)

Month
2     131.610588
3     118.370323
6     116.870508
1     114.079677
4      97.742593
12     96.592115
7      92.214337
11     86.599000
8      85.797491
5      84.314839
9      79.367778
10     68.356989
dtype: float64

Т.к. ответ не сошелся проверим, может имеется ввиду месяц в котором статистически чаше всего встречаются дни с дождями, а не количество освков в среднем в день макстимально:

In [527]:
month_rain = df.groupby(['Month'])['RainToday'].sum() / df.groupby(['Month'])['Date'].nunique()
month_rain.sort_values(ascending=False)

Month
7     11.430108
6     11.074576
8     10.673835
9      9.629630
5      9.358065
3      9.132258
4      9.077778
2      8.533333
10     8.318996
12     8.258065
11     8.050000
1      7.893548
dtype: float64

In [528]:
df = df.drop(['Date'], axis=1)

In [529]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df['Month'] = df['Month'].astype('category')
dum_df = pd.get_dummies(df[categoricals])
df = df.drop(categoricals, axis=1)
df = pd.concat([df,dum_df], axis=1)
df

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,0,0,0,0,0,0,0,1,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,0,0,0,0,0,0,0,0,1
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,...,0,0,0,0,0,0,0,0,0,0
145456,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,...,0,0,0,0,0,0,0,0,0,0
145457,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,...,0,0,0,0,0,0,0,0,1,0
145458,7.8,27.0,0.0,28.0,13.0,7.0,51.0,24.0,1019.4,1016.5,...,0,0,0,0,0,0,0,0,0,0


In [530]:
df = df.dropna()
df.shape

(78257, 124)

In [531]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=31)

y_test.mean()

0.22770253002811142

Для практики реализуем бутстреп вручную. Сделаем оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки (то есть для среднего значения по признаку MinTemp)

In [532]:
np.random.seed(31)

In [533]:
#Генерируем массив из индексов от 0 до длины данных Х, из 100 элементов длины обучающей выборки
# - это будут наши случайные индексы бутстреп выборок данных
butstrep_ind_vectors = np.random.randint(0,y.shape[0], size=(1000,y_train.shape[0]))

In [534]:
bustrep_value_vectors = []

for ind_vector in butstrep_ind_vectors:
    bustrep_value_vectors.append(X.iloc[ind_vector]['MinTemp'].mean()) #Просто добавляем в список среднее по 'MinTemp' по сгенерированным интексам
    
np.std(bustrep_value_vectors).round(2)

0.03

Логистическая регерссия

In [537]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [539]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_test_pred = lr.predict(X_test)
print('Метрика roc_aug по модели Логистической регрессиии =', roc_auc_score(y_test, y_test_pred).round(2) )

Метрика roc_aug по модели Логистической регрессиии = 0.73


c:\Users\killr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Дерево решений сразу с подбором гиперпараметров

In [540]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}
dc = DecisionTreeClassifier(random_state=42)

BestDC = GridSearchCV(
    estimator=dc,
    param_grid=params,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1
)

BestDC.fit(X_train, y_train)

y_test_pred = BestDC.predict(X_test)
print('Метрика roc_aug по модели Дерева решений =', roc_auc_score(y_test, y_test_pred).round(2) )

BestDC.best_params_


Метрика roc_aug по модели Дерева решений = 0.7


{'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}

Случайный лес

In [542]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=31, n_estimators=100)
rf.fit(X_train, y_train)

y_test_pred = rf.predict(X_test)
print('Метрика roc_aug по модели слечайного леса на 100 деревьях =', roc_auc_score(y_test, y_test_pred).round(2) )


Метрика roc_aug по модели слечайного леса на 100 деревьях = 0.73


Подбор гиперпараметров для случайного леса

In [544]:
params = {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}

BestRF = GridSearchCV(
    estimator=RandomForestClassifier(random_state=31, n_estimators=100),
    param_grid=params,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1
)

BestRF.fit(X_train, y_train)

y_test_pred = BestRF.predict(X_test)
print('Метрика roc_aug по модели  случайного леса на оптимальных параметрах', roc_auc_score(y_test, y_test_pred).round(2) )

BestRF.best_params_

Метрика roc_aug по модели  случайного леса на оптимальных параметрах 0.7


{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 3}

In [548]:
rf_best = RandomForestClassifier(max_depth= 15, max_features= 7, min_samples_leaf= 3, random_state=31, n_estimators=100)
rf_best.fit(X_train,y_train)

pd.Series(rf_best.feature_importances_,index=X_train.columns).sort_values(ascending=False)

Humidity3pm            0.250783
Rainfall               0.079757
Humidity9am            0.070403
Cloud9am               0.067092
Pressure3pm            0.065272
                         ...   
Location_Newcastle     0.000000
Location_SalmonGums    0.000000
Location_Nhil          0.000000
Location_NorahHead     0.000000
Location_GoldCoast     0.000000
Length: 123, dtype: float64